In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "drive/My Drive/recvis20_a3/"
# %pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
%pip install logger
%pip install numpy
%pip install torch
%pip install torchvision
%pip install tqdm
%pip install timm
%pip install tensorboard

[Errno 2] No such file or directory: 'drive/My Drive/recvis20_a3/'
/content/drive/My Drive/recvis20_a3


In [ ]:
import argparse
import json
import os
import glob
import time
from PIL import Image
from utils.utils import *
import numpy as np
import torch.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from tqdm import tqdm_notebook
from model import *
from early_stopping.pytorchtools import EarlyStopping
from torch.utils.tensorboard import SummaryWriter
import pdb
from uuid import uuid4

path = os.getcwd()
path_id = os.path.join("experiment", '31d84')
path_load_model = os.path.join(path_id, 'model.pt')
path_data = os.path.join(path, 'cropped_NAbirds')
path_save = os.path.join(path, 'filtered_cropped_NAbirds')

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
model = load_model(path_load_model, 'vit', 'vit_large_patch16_224', use_cuda=True, load_weights=1)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

[16/Nov/2020 09:42:16] INFO - vit model loaded
Using GPU


In [ ]:
n = 9
model.eval()
os.makedirs(path_save + '/subsample{}'.format(n), exist_ok=False)
path_imgs = glob.glob(path_data + '/subsample{}/*.jpg'.format(n))

saved = 0
for file in tqdm_notebook(path_imgs):
  
  img = pil_loader(file)
  data = data_transforms(img)
  data = data.unsqueeze(0)
  if torch.cuda.is_available():
    data = data.cuda()
  output = model(data)
  out = F.softmax(output, dim=-1)

  if torch.max(out, dim=-1)[0].item() > 0.8:
    saved += 1
    print('\n' + str(saved))

    basename = os.path.basename(file)
    img.save(path_save + '/subsample{}/'.format(n) + basename, "JPEG")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61
